# Setup

In [14]:
REDIS_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

REDIS_TOTAL_NODES = 6
REDIS_NODE_IPS = ["10.15.20.2"] * REDIS_TOTAL_NODES
REDIS_NODE_NAMES = [f"redis-node-{i+1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_HOSTNAMES = [
    f"{REDIS_NODE_NAMES[i]}.mavasbel.vpn.itam.mx" for i in range(REDIS_TOTAL_NODES)
]
REDIS_NODE_PORTS = [f"{6380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_BUS_PORTS = [f"{16380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]

REDIS_WORKDIR = "/data"

REDIS_ADMIN_PASSWORD = "redis"
REDIS_DEFAULT_PASSWORD = "redis"
REDIS_INIT_USER = "redis"
REDIS_INIT_PASSWORD = "redis"

In [15]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Session creation

In [16]:
import pprint
from redis.cluster import RedisCluster, ClusterNode
from redis.cluster import LoadBalancingStrategy

redis_nodes = [
    ClusterNode(f"{REDIS_NODE_HOSTNAMES[i]}", REDIS_NODE_PORTS[i])
    for i in range(0, REDIS_TOTAL_NODES)
]
pprint.pprint(f"🔗 Connecting to: {redis_nodes}")

try:
    rc = RedisCluster(
        startup_nodes=redis_nodes,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,  # decode_responses=True converts bytes to strings automatically
        load_balancing_strategy=LoadBalancingStrategy.RANDOM_REPLICA,
        search_all_nodes=True,
        # read_from_replicas=True,
        # dynamic_startup_nodes=False,
        require_full_coverage=False,
    )
    cluster_status = rc.cluster_info()["cluster_state"]
    nodes_count = len(rc.get_nodes())
    state_icon = "🟢" if cluster_status == "ok" else "🔴"
    print("\n✅ Cluster connected")
    print(f"{state_icon} Cluster State: {cluster_status.upper()}")
    print(f"🌐 Nodes Discovered: {nodes_count}")
except Exception as e:
    print(f"Connection failed: {e}")

('🔗 Connecting to: '
 '[[host=redis-node-1.mavasbel.vpn.itam.mx,port=6381,name=redis-node-1.mavasbel.vpn.itam.mx:6381,server_type=None,redis_connection=None], '
 '[host=redis-node-2.mavasbel.vpn.itam.mx,port=6382,name=redis-node-2.mavasbel.vpn.itam.mx:6382,server_type=None,redis_connection=None], '
 '[host=redis-node-3.mavasbel.vpn.itam.mx,port=6383,name=redis-node-3.mavasbel.vpn.itam.mx:6383,server_type=None,redis_connection=None], '
 '[host=redis-node-4.mavasbel.vpn.itam.mx,port=6384,name=redis-node-4.mavasbel.vpn.itam.mx:6384,server_type=None,redis_connection=None], '
 '[host=redis-node-5.mavasbel.vpn.itam.mx,port=6385,name=redis-node-5.mavasbel.vpn.itam.mx:6385,server_type=None,redis_connection=None], '
 '[host=redis-node-6.mavasbel.vpn.itam.mx,port=6386,name=redis-node-6.mavasbel.vpn.itam.mx:6386,server_type=None,redis_connection=None]]')

✅ Cluster connected
🟢 Cluster State: OK
🌐 Nodes Discovered: 3


In [17]:
import pprint
import pandas as pd
from IPython.display import display

try:
    print("🛰️ Cluster Nodes View")
    display(pd.DataFrame(rc.execute_command("CLUSTER NODES")).transpose().sort_index())
except Exception as e:
    pprint.pprint(f"Error checking survivors: {e}")

🛰️ Cluster Nodes View


,node_id,hostname,flags,master_id,last_ping_sent,last_pong_rcvd,epoch,slots,migrations,connected
10.15.20.2:6381,af43eefdb2b58581e8e5b6846a7dd1c09cec2997,,"myself,master",-,0,0,1,"[[0, 5460]]",[],True
10.15.20.2:6382,4c9170f945ec7584365ef987956990b7f13382cf,,master,-,0,1768032225515,2,"[[5461, 10922]]",[],True
10.15.20.2:6383,3d02993b440e32b6164253ae5ccee436b7b01504,,master,-,0,1768032225515,3,"[[10923, 16383]]",[],True
10.15.20.2:6384,20c16ba078ea7d88b657db438656e56f055bbb48,,slave,4c9170f945ec7584365ef987956990b7f13382cf,0,1768032224000,2,[],[],True
10.15.20.2:6385,61f2e4c5e294f2c31976da2795f0d949e324db33,,slave,3d02993b440e32b6164253ae5ccee436b7b01504,0,1768032224492,3,[],[],True
10.15.20.2:6386,d14990b45d8303ef631e834c108b83e25ba4e776,,slave,af43eefdb2b58581e8e5b6846a7dd1c09cec2997,0,1768032225000,1,[],[],True


In [18]:
import pprint

try:
    print("📈 Cluster Info")
    pprint.pprint(rc.execute_command("CLUSTER INFO"))
except Exception as e:
    pprint.pprint(f"Error checking survivors: {e}")

📈 Cluster Info
{'cluster_current_epoch': '6',
 'cluster_known_nodes': '6',
 'cluster_my_epoch': '1',
 'cluster_size': '3',
 'cluster_slot_migration_active_tasks': '0',
 'cluster_slot_migration_active_trim_current_job_keys': '0',
 'cluster_slot_migration_active_trim_current_job_trimmed': '0',
 'cluster_slot_migration_active_trim_running': '0',
 'cluster_slot_migration_stats_active_trim_cancelled': '0',
 'cluster_slot_migration_stats_active_trim_completed': '0',
 'cluster_slot_migration_stats_active_trim_started': '0',
 'cluster_slots_assigned': '16384',
 'cluster_slots_fail': '0',
 'cluster_slots_ok': '16384',
 'cluster_slots_pfail': '0',
 'cluster_state': 'ok',
 'cluster_stats_messages_meet_received': '5',
 'cluster_stats_messages_ping_received': '141',
 'cluster_stats_messages_ping_sent': '140',
 'cluster_stats_messages_pong_received': '140',
 'cluster_stats_messages_pong_sent': '146',
 'cluster_stats_messages_received': '286',
 'cluster_stats_messages_sent': '286',
 'total_cluster_li

In [19]:
from redis import Redis
import pandas as pd

results = []
for i in range(REDIS_TOTAL_NODES):
    try:
        rn = Redis(
            host=f"{REDIS_NODE_HOSTNAMES[i]}",
            port=REDIS_NODE_PORTS[i],
            username=REDIS_INIT_USER,
            password=REDIS_INIT_PASSWORD,
            decode_responses=True,
        )
        ann_ip = rn.config_get("cluster-announce-ip")["cluster-announce-ip"]
        ann_port = rn.config_get("cluster-announce-port")["cluster-announce-port"]
        role = rn.info("replication")["role"]
        results.append(
            {
                "Port": REDIS_NODE_PORTS[i],
                "Role": role,
                "Announced IP": ann_ip,
                "Announced Port": ann_port,
            }
        )
    except Exception as e:
        results.append(
            {
                "Port": REDIS_NODE_PORTS[i],
                "Role": "OFFLINE",
                "Announced IP": "-",
                "Announced Port": "-",
            }
        )

print(pd.DataFrame(results))

   Port    Role Announced IP Announced Port
0  6381  master   10.15.20.2           6381
1  6382  master   10.15.20.2           6382
2  6383  master   10.15.20.2           6383
3  6384   slave   10.15.20.2           6384
4  6385   slave   10.15.20.2           6385
5  6386   slave   10.15.20.2           6386


In [20]:
from typing import cast
from redis import Redis

for node in rc.get_nodes():
    try:
        node = cast(ClusterNode, node)
        client = Redis(
            host=node.host,
            port=node.port,
            username=REDIS_INIT_USER,
            password=REDIS_INIT_PASSWORD,
            decode_responses=True,
        )
        role = client.info("replication")["role"]
        connected_slaves = client.info("replication").get("connected_slaves", 0)
        print(
            f"Node {node.port} ({role.upper()}) has {connected_slaves} slaves connected."
        )
    except Exception as e:
        pprint.pprint(f"Error: {e}")

Node 6381 (MASTER) has 1 slaves connected.
Node 6382 (MASTER) has 1 slaves connected.
Node 6383 (MASTER) has 1 slaves connected.


In [21]:
from collections import Counter
from typing import cast
from redis import Redis

# 1. Populate a test key
test_keys = [f"key_{i}" for i in range(10)]
for k in test_keys:
    rc.set(k, "data")  # Ensure keys exist

# 2. Perform 500 reads and track which node handles each
node_hits = []
for i in range(500):
    node = cast(ClusterNode, node)
    node = cast(ClusterNode, rc.get_node_from_key(test_keys[i % REDIS_TOTAL_NODES]))
    node_hits.append(f"{node.host}:{node.port}")
    rc.get(test_keys[i % REDIS_TOTAL_NODES])

# 3. Display the distribution
print("📊 Read Distribution for a Single Key")
counts = Counter(node_hits)
for node, count in counts.items():
    # Identify if it's a master or slave for context
    role = Redis(
        host=node.split(":")[0],
        port=node.split(":")[1],
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
    ).info("replication")["role"]
    print(f"Node {node} ({role.upper()}): {count} hits")

📊 Read Distribution for a Single Key
Node 10.15.20.2:6383 (MASTER): 334 hits
Node 10.15.20.2:6382 (MASTER): 83 hits
Node 10.15.20.2:6381 (MASTER): 83 hits


In [22]:
from redis import Redis

# 1. Reset statistics on all nodes
print("Emptying stats on all nodes...")
for node in rc.get_nodes():
    node = cast(ClusterNode, node)
    r_internal = Redis(
        host=node.host,
        port=node.port,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    r_internal.config_resetstat()

# 2. Perform a burst of reads on multiple keys
print("Performing 500 reads across 10 keys...")
test_keys = [f"key_{i}" for i in range(10)]
for k in test_keys:
    rc.set(k, "data")  # Ensure keys exist

for _ in range(50):
    for k in test_keys:
        rc.get(k)

# 3. Check who actually processed the 'GET' command
print("\n📊 ACTUAL Execution Stats (From Redis Engines)")
for node in rc.get_nodes():
    node = cast(ClusterNode, node)

    r_internal = Redis(
        host=node.host,
        port=node.port,
        username=REDIS_INIT_USER,
        password=REDIS_INIT_PASSWORD,
        decode_responses=True,
    )
    role = r_internal.info("replication")["role"]
    stats = r_internal.info("commandstats")
    get_calls = stats.get("cmdstat_get", {}).get("calls", 0)

    if get_calls > 0:
        print(f"✅ Node {node.port} ({role.upper()}): {get_calls} GETs handled")
    else:
        print(f"❌ Node {node.port} ({role.upper()}): 0 GETs handled")

Emptying stats on all nodes...
Performing 500 reads across 10 keys...

📊 ACTUAL Execution Stats (From Redis Engines)
✅ Node 6381 (MASTER): 100 GETs handled
✅ Node 6382 (MASTER): 100 GETs handled
✅ Node 6383 (MASTER): 300 GETs handled


In [23]:
from redis import Redis
import pandas as pd

results = []
for i in range(REDIS_TOTAL_NODES):
    try:
        rn = Redis(
            host=REDIS_NODE_HOSTNAMES[i],
            port=REDIS_NODE_PORTS[i],
            username=REDIS_INIT_USER,
            password=REDIS_INIT_PASSWORD,
            decode_responses=True,
        )
        info = rn.execute_command("CLUSTER INFO")

        results.append(
            {
                "Port": REDIS_NODE_PORTS[i],
                "Known Nodes": info.get("cluster_known_nodes"),
                "Pings Sent": info.get("cluster_stats_messages_ping_sent"),
                "Pongs Received": info.get("cluster_stats_messages_pong_received"),
                "State": info.get("cluster_state"),
            }
        )
    except Exception as e:
        results.append({"Port": REDIS_NODE_PORTS[i], "State": "OFFLINE"})

print(pd.DataFrame(results))

   Port Known Nodes Pings Sent Pongs Received State
0  6381           6        141            141    ok
1  6382           6        140            141    ok
2  6383           6        140            141    ok
3  6384           6        150            151    ok
4  6385           6        145            146    ok
5  6386           6        141            142    ok


In [24]:
# import time
# from redis import Redis

# node = Redis(
#     host=REDIS_NODE_HOSTNAMES[0],
#     port=REDIS_NODE_PORTS[0],
#     username=REDIS_INIT_USER,
#     password=REDIS_INIT_PASSWORD,
#     decode_responses=True,
# )

# print("👀 Monitoring Cluster for Failover Events... (Kill your master now)")
# for _ in range(30):
#     nodes = node.execute_command("CLUSTER NODES")
#     # Look for the 'fail' flag or 'failover' state
#     for line in str(nodes).split("\n"):
#         if "fail" in line or "handshake" in line:
#             print(f"⏰ {time.strftime('%H:%M:%S')} | Event Detected: {line}...")
#     time.sleep(2)

In [25]:
# from redis import Redis

# replica = Redis(
#     host=REDIS_NODE_HOSTNAMES[3],
#     port=REDIS_NODE_PORTS[3],
#     username=REDIS_INIT_USER,
#     password=REDIS_INIT_PASSWORD,
#     decode_responses=True,
# )

# try:
#     print(f"🚀 Attempting forced takeover on port {replica_port}...")
#     # 'TAKEOVER' is the aggressive version of failover
#     replica.execute_command("CLUSTER FAILOVER TAKEOVER")
#     print("✅ Success! The replica has promoted itself to MASTER.")
#     rc.nodes_manager.initialize()
#     print("✅ Success! The cluster has been initialized.")
# except Exception as e:
#     print(f"❌ Manual takeover failed: {e}")
#     print("Check if the Replica can actually 'see' the other Masters.")